In [2]:
import pandas as pd
import requests
# import pickle
import sqlalchemy

In [9]:
# Retrieve Mentions Data as DataFrame
db_name = "SEC_Filings.db"
table_name = "10K_Data"

engine = sqlalchemy.create_engine('sqlite:///' + db_name, execution_options={"sqlite_raw_colnames": True})
df = pd.read_sql_table(table_name, engine)

df = df.drop(['SIC', 'Filing_Date', 'Filing_Link', 'Accession_No', 'Filing_Document_Text'], axis=1)

,CIK,Company
0,0001527383,BankGuam Holding Co
1,0001527383,BankGuam Holding Co
2,0001527383,BankGuam Holding Co
3,0001527383,BankGuam Holding Co
4,0001527383,BankGuam Holding Co
...,...,...
3912,0001488419,"Sino Agro Food, Inc."
3913,0001488419,"Sino Agro Food, Inc."
3914,0001488419,"Sino Agro Food, Inc."
3915,0001488419,"Sino Agro Food, Inc."


In [47]:
pd.set_option('display.max_rows', None)

headers = {'User-Agent': "evanmcgarry7@gmail.com"}

tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)

tickers_cik = pd.json_normalize(pd.json_normalize(tickers_cik.json(), max_level=0).values[0])
tickers_cik["cik_str"] = tickers_cik["cik_str"].astype(str).str.zfill(10)
tickers_cik.columns= ['CIK', 'Ticker', 'Company_Name']
tickers_cik.drop(['Company_Name'], axis=1, inplace=True)

tickers_cik['CIK'].astype(int)
df['CIK'].astype(int)

df_full = df.merge(tickers_cik, how='left', on='CIK')
df_full.loc[df_full['Ticker'].isna()]
df_full.dropna(inplace=True)
df_full.reset_index(drop=True, inplace=True)
df_full = df_full.drop_duplicates()

tickers_list = []
for company in df_full['Company'].unique():
    tickers_list.append(df_full[df_full['Company']==company].sort_values(by='Ticker').head(1).values.squeeze())

df_tickers = pd.DataFrame(tickers_list, columns=['CIK', 'Company', 'Ticker'])

In [49]:
# Insert Tickers Data into Table in Database
db_name2 = "SEC_Filings.db"
table_name2 = "Ticker_Table"

engine2 = sqlalchemy.create_engine('sqlite:///' + db_name, execution_options={"sqlite_raw_colnames": True})
df_tickers.to_sql(table_name2, engine2, if_exists='replace', index=False)

532